In [4]:
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset

from VAEmodel import *

In [5]:
# Load the processed MSA data
import pickle
from pathlib import Path

DATA_DIR = Path("../../data")
REF_SEQ_ID = "PF00041"
AA_TYPES = ['R', 'H', 'K',
      'D', 'E',
      'S', 'T', 'N', 'Q',
      'C', 'G', 'P',
      'A', 'V', 'I', 'L', 'M', 'F', 'Y', 'W']

enumd_msa = np.load(DATA_DIR / "processed" / "enumd_mtx_{}.npy".format(REF_SEQ_ID))
print("enumd_msa.shape: ", enumd_msa.shape)
seq_weights = np.load(DATA_DIR / "processed" / "seq_weights_{}.npy".format(REF_SEQ_ID))
print("seq_weights.shape: ", seq_weights.shape)

msa = MSA_Dataset(enumd_msa, seq_weights, np.array(AA_TYPES), MSA_to_OneHot)

# check length = num of seqs
print("Number of sequences in loaded alignment: ", len(msa))
print("Length of loaded alignment (final number of columns/a.a. positions in alignment for each sequence): ", enumd_msa.shape[1])

enumd_msa.shape:  (320066, 79)
seq_weights.shape:  (320066,)
Number of sequences in loaded alignment:  320066
Length of loaded alignment (final number of columns/a.a. positions in alignment for each sequence):  79


In [7]:
# import torch_xla.core.xla_model as xm
# dev = xm.xla_device()

print("GPU available: ", torch.cuda.is_available())
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 2-dimensional latent space
vae = VAE(len(AA_TYPES), 2, enumd_msa.shape[1] * len(AA_TYPES), [200, 200])
vae.to(dev)

GPU available:  True


VAE(
  (encoder_comm_layers): Sequential(
    (0): Sequential(
      (0): Linear(in_features=2, out_features=200, bias=True)
      (1): Tanh()
    )
    (1): Sequential(
      (0): Linear(in_features=200, out_features=200, bias=True)
      (1): Tanh()
    )
  )
  (enc_mu): Linear(in_features=200, out_features=2, bias=True)
  (enc_logvars): Linear(in_features=200, out_features=2, bias=True)
)

In [8]:

vae.encode(msa[0][0].to(dev))

AttributeError: 'numpy.ndarray' object has no attribute 'to'